In [25]:
import os
import random
import sys

import git
import numpy as np
from gym import spaces

%matplotlib inline
from matplotlib import pyplot as plt

from PIL import Image

import habitat
from habitat.core.logging import logger
from habitat.core.registry import registry
from habitat.sims.habitat_simulator.actions import HabitatSimActions
from habitat.tasks.nav.nav import NavigationTask
from habitat_baselines.common.baseline_registry import baseline_registry
from habitat_baselines.config.default import get_config as get_baselines_config

In [26]:
if __name__ == "__main__":
    config = habitat.get_config(
        config_path="benchmark/rearrange/pick.yaml",
        overrides=[
            "habitat.environment.max_episode_steps=20",
            "habitat.environment.iterator_options.shuffle=False",
        ],
    )

    try:
        env.close()  # type: ignore[has-type]
    except NameError:
        pass
    env = habitat.Env(config=config)

2024-01-17 16:35:14,150 Initializing dataset RearrangeDataset-v0
Exception ignored in: <function Simulator.__del__ at 0x7f4a5a225430>
Traceback (most recent call last):
  File "/home/anand/miniconda3/envs/habitat/lib/python3.9/site-packages/habitat_sim-0.2.5-py3.9-linux-x86_64.egg/habitat_sim/simulator.py", line 528, in __del__
    self.close(destroy=True)
  File "/home/anand/miniconda3/envs/habitat/lib/python3.9/site-packages/habitat_sim-0.2.5-py3.9-linux-x86_64.egg/habitat_sim/simulator.py", line 131, in close
    if not self._initialized:
AttributeError: 'RearrangeSim' object has no attribute '_initialized'
2024-01-17 16:35:44,603 initializing sim RearrangeSim-v0
[16:35:44:730861]:[Warning]:[Metadata] SceneDatasetAttributesManager.cpp(206)::setValsFromJSONDoc : (Articulated Object) : No Glob path result for `2024-01-17 16:35:48,927 Initializing task RearrangePickTask-v0
data/replica_cad/../hab_fetch_1.0/robots/fetch_no_base.urdf`
[16:35:44:813690]:[Warning]:[Metadata] SceneDatasetAt

Renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 545.23.08
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_texture_view
    GL_ARB_framebuffer_no_attachments
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
    GL_KHR_parallel_shader_compile
    GL_NV_depth_buffer_float
Using driver workarounds:
    no-forward-compatible-core-context
    nv-egl-incorrect-gl11-function-pointers
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits



[16:35:44:813784]:[Warning]:[Metadata] SceneDatasetAttributesManager.cpp(267)::loadAndValidateMap : `navmesh_instances` Value : `navmeshes/v3_sc4_staging_04.navmesh` not found on disk as absolute path or relative to `data/replica_cad`
[16:35:44:813803]:[Warning]:[Metadata] SceneDatasetAttributesManager.cpp(267)::loadAndValidateMap : `navmesh_instances` Value : `navmeshes/v3_sc4_staging_05.navmesh` not found on disk as absolute path or relative to `data/replica_cad`
[16:35:44:813821]:[Warning]:[Metadata] SceneDatasetAttributesManager.cpp(267)::loadAndValidateMap : `navmesh_instances` Value : `navmeshes/v3_sc4_staging_06.navmesh` not found on disk as absolute path or relative to `data/replica_cad`
[16:35:44:813840]:[Warning]:[Metadata] SceneDatasetAttributesManager.cpp(267)::loadAndValidateMap : `navmesh_instances` Value : `navmeshes/v3_sc4_staging_07.navmesh` not found on disk as absolute path or relative to `data/replica_cad`
[16:35:44:813858]:[Warning]:[Metadata] SceneDatasetAttribut

In [27]:
env.number_of_episodes=[3]

In [53]:
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations.utils import (
    observations_to_image,
    overlay_frame,
)
from habitat.tasks.rearrange.utils import (
    CollisionDetails,
    UsesArticulatedAgentInterface,
    batch_transform_point,
    get_angle_to_pos,
    rearrange_logger,
)
from habitat.tasks.rearrange.rearrange_sim import RearrangeSim
observations = env.reset()  # noqa: F841

print("Agent acting inside environment.")
count_steps = 0
# To save the video
video_file_path = "data/example_interact.mp4"
video_writer = vut.get_fast_video_writer(video_file_path, fps=30)

while not env.episode_over:
    observations = env.step(env.action_space.sample())  # noqa: F841
    info = env.get_metrics()
    print(info)
    # print(observations)
    render_obs = observations_to_image(observations, info)
    render_obs = overlay_frame(render_obs, info)

    video_writer.append_data(render_obs)

    count_steps += 1
print("Episode finished after {} steps.".format(count_steps))

video_writer.close()
if vut.is_notebook():
    vut.display_video(video_file_path)

Agent acting inside environment.
{'articulated_agent_force': {'accum': 0.0, 'instant': 9.787683375179768}, 'force_terminate': False, 'robot_collisions': {'total_collisions': 0, 'robot_obj_colls': 0, 'robot_scene_colls': 0, 'obj_scene_colls': 0}, 'collision_terminate': False, 'ee_to_rest_distance': 0.5758212, 'ee_to_object_distance': {'0': 0.75780064}, 'did_pick_object': 0, 'pick_success': False, 'pick_reward': -0.00800000037997961, 'did_violate_hold_constraint': False, 'num_steps': 1}
{'articulated_agent_force': {'accum': 0.0, 'instant': 10.699151083827019}, 'force_terminate': False, 'robot_collisions': {'total_collisions': 0, 'robot_obj_colls': 0, 'robot_scene_colls': 0, 'obj_scene_colls': 0}, 'collision_terminate': False, 'ee_to_rest_distance': 0.5705843, 'ee_to_object_distance': {'0': 0.7553139}, 'did_pick_object': 0, 'pick_success': False, 'pick_reward': 0.004000000189989805, 'did_violate_hold_constraint': False, 'num_steps': 2}
{'articulated_agent_force': {'accum': 0.0, 'instant':

In [55]:
from habitat.tasks.nav.nav import PointGoalSensor
class MultiObjSensor(PointGoalSensor):
    """
    Abstract parent class for a sensor that specifies the locations of all targets.
    """

    def __init__(self, *args, task, **kwargs):
        self._task = task
        self._sim: RearrangeSim
        super().__init__(*args, task=task, **kwargs)

    def _get_observation_space(self, *args, **kwargs):
        n_targets = self._task.get_n_targets()
        return spaces.Box(
            shape=(n_targets * 3,),
            low=np.finfo(np.float32).min,
            high=np.finfo(np.float32).max,
            dtype=np.float32,
        )

class TargetStartSensor(UsesArticulatedAgentInterface, MultiObjSensor):
    """
    Relative position from end effector to target object
    """

    cls_uuid: str = "obj_start_sensor"

    def get_observation(self, *args, observations, episode, **kwargs):
        self._sim: RearrangeSim
        global_T = self._sim.get_agent_data(
            self.agent_id
        ).articulated_agent.ee_transform()
        T_inv = global_T.inverted()
        pos = self._sim.get_target_objs_start()
        return batch_transform_point(pos, T_inv, np.float32).reshape(-1)

In [39]:
observations.keys()
# print(observations["ee_pos"])
# print(observations["relative_resting_position"])

dict_keys(['head_depth', 'head_rgb', 'obj_start_sensor', 'joint', 'is_holding', 'ee_pos', 'relative_resting_position'])

In [29]:
def display_sample(
    rgb_obs, semantic_obs=np.array([]), depth_obs=np.array([])):  # noqa: B006
    from habitat_sim.utils.common import d3_40_colors_rgb

    rgb_img = Image.fromarray(rgb_obs, mode="RGB")

    arr = [rgb_img]
    titles = ["rgb"]
    if semantic_obs.size != 0:
        semantic_img = Image.new(
            "P", (semantic_obs.shape[1], semantic_obs.shape[0])
        )
        semantic_img.putpalette(d3_40_colors_rgb.flatten())
        semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
        semantic_img = semantic_img.convert("RGBA")
        arr.append(semantic_img)
        titles.append("semantic")

    if depth_obs.size != 0:
        depth_img = Image.fromarray(
            (depth_obs / 10 * 255).astype(np.uint8), mode="L"
        )
        arr.append(depth_img)
        titles.append("depth")

    plt.figure(figsize=(12, 8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i + 1)
        ax.axis("off")
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show(block=False)

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from gym import spaces

ACTION_SPACE = spaces.Discrete(4)

OBSERVATION_SPACES = {
    "depth_model": spaces.Dict(
        {
            "depth": spaces.Box(
                low=0,
                high=1,
                shape=(224, 224, 1),
                dtype=np.float32,
            ),
            "pointgoal_with_gps_compass": spaces.Box(
                low=np.finfo(np.float32).min,
                high=np.finfo(np.float32).max,
                shape=(2,),
                dtype=np.float32,
            ),
        }
    )
}
class MyPolicy(nn.Module):
    def __init__(self, observation_space, action_space, hidden_size=512, num_recurrent_layers=2, rnn_type="LSTM", resnet_baseplanes=32, backbone="resnet18", normalize_visual_inputs=False, force_blind_policy=False, policy_config=None, aux_loss_config={}, fuse_keys=None, **kwargs):
        super(MyPolicy, self).__init__()

        # Define the components of the policy
        self.conv = nn.Conv2d(observation_space['depth'].shape[2], resnet_baseplanes, kernel_size=3, stride=2, padding=1)
        self.resnet = models.resnet18(pretrained=True)  # Assuming you want to use a pretrained ResNet18
        self.fc1 = nn.Linear(resnet_baseplanes * 4 * 4 + 2, hidden_size)
        self.rnn = getattr(nn, rnn_type)(hidden_size, hidden_size, num_recurrent_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, action_space.n)  # Use action_space.n instead of action_space

    def forward(self, obs, rnn_hidden_states=None, masks=None):
        depth_image = obs['head_depth']
        pointgoal_with_gps_compass = obs['relative_resting_position']

        # Process the depth image through the convolutional layers
        x = F.relu(self.conv(depth_image))
        x = F.avg_pool2d(x, 4)  # Assuming the output size is 4x4
        x = x.view(x.size(0), -1)  # Flatten the tensor

        # Process the pointgoal_with_gps_compass
        x = torch.cat((x, pointgoal_with_gps_compass), dim=1)

        # Pass through fully connected and recurrent layers
        x = F.relu(self.fc1(x))
        if rnn_hidden_states is not None:
            x, rnn_hidden_states = self.rnn(x.unsqueeze(0), rnn_hidden_states)
            x = x.squeeze(0)
        else:
            x, rnn_hidden_states = self.rnn(x.unsqueeze(0))
            x = x.squeeze(0)

        # Output the action logits
        action_logits = self.fc2(x)

        return action_logits, rnn_hidden_states

# Create an instance of MyPolicy
MODELS = {
    "pointnav_weights.pth": {
        "backbone": "resnet18",
        "observation_space": OBSERVATION_SPACES["depth_model"],
        "action_space": ACTION_SPACE,
    }}
model_weights_path = 'data/new_checkpoints/pointnav_weights.pth'  # Replace with the actual path to your model weights file

pretrained_state = torch.load(model_weights_path, map_location="cpu")

observation_space = {'depth': torch.rand(1, 224, 224, 1), 'pointgoal_with_gps_compass': torch.rand(1, 2)}
action_space = spaces.Discrete(4)
policy = MyPolicy(observation_space=OBSERVATION_SPACES["depth_model"], action_space=ACTION_SPACE)

policy.load_state_dict(pretrained_state)

# Set the model to evaluation mode
policy.eval()
obs = env.reset()  

action_logits, rnn_hidden_states = policy(obs)
print(action_logits)





action = None
obs = env.reset()
# print(obs["head_rgb"])
valid_actions = ["stop","move_forward","turn_left", "turn_right"]
interactive_control = False  # @param {type:"boolean"}
while action != "stop":
    display_sample(obs["head_rgb"])
    print(
        "distance to goal: {:.2f}".format(
            obs["ee_pos"][0]
        )
    )
    print(
        "angle to goal (radians): {:.2f}".format(
            obs["ee_pos"][1]
        )
    )
    obs = env.step(
        {
            "action": action,
        }
    )

env.close()

RuntimeError: Error(s) in loading state_dict for MyPolicy:
	Missing key(s) in state_dict: "conv.weight", "conv.bias", "resnet.conv1.weight", "resnet.bn1.weight", "resnet.bn1.bias", "resnet.bn1.running_mean", "resnet.bn1.running_var", "resnet.layer1.0.conv1.weight", "resnet.layer1.0.bn1.weight", "resnet.layer1.0.bn1.bias", "resnet.layer1.0.bn1.running_mean", "resnet.layer1.0.bn1.running_var", "resnet.layer1.0.conv2.weight", "resnet.layer1.0.bn2.weight", "resnet.layer1.0.bn2.bias", "resnet.layer1.0.bn2.running_mean", "resnet.layer1.0.bn2.running_var", "resnet.layer1.1.conv1.weight", "resnet.layer1.1.bn1.weight", "resnet.layer1.1.bn1.bias", "resnet.layer1.1.bn1.running_mean", "resnet.layer1.1.bn1.running_var", "resnet.layer1.1.conv2.weight", "resnet.layer1.1.bn2.weight", "resnet.layer1.1.bn2.bias", "resnet.layer1.1.bn2.running_mean", "resnet.layer1.1.bn2.running_var", "resnet.layer2.0.conv1.weight", "resnet.layer2.0.bn1.weight", "resnet.layer2.0.bn1.bias", "resnet.layer2.0.bn1.running_mea